# Table of Contents
* [TP2 - Discrimination de bonnes ou mauvaises réponses dans une base de questions-réponses.](#TP2---Discrimination-de-bonnes-ou-mauvaises-réponses-dans-une-base-de-questions-réponses.)
	* [Première partie : récupérer les données](#Première-partie-:-récupérer-les-données)
		* [Récupération sur plusieurs années](#Récupération-sur-plusieurs-années)
	* [Deuxième partie: Trouver et définir des features](#Deuxième-partie:-Trouver-et-définir-des-features)
		* [Lecture de données](#Lecture-de-données)
		* [Construction de variables explicatives ](#Construction-de-variables-explicatives)
	* [Troisième partie: discrimination](#Troisième-partie:-discrimination)
		* [Première question](#Première-question)
		* [Seconde question](#Seconde-question)
		* [Tertio ](#Tertio)


In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
%%HTML
<style>
em {
    color: green;
}
strong
{
    color: blue;
}
</style>

# TP2 (sujet) - Discrimination de bonnes ou mauvaises réponses dans une base de questions-réponses.

**Installation**: Vous téléchargerez le fichier [install_tp2](install_tp2)  dans un répertoire local qui vous va bien. Ceci étant fait, vous ouvrez un terminal, vous rendez dans ce répertoire, changez les droits d'accès au fichier par `chmod 755 install_tp2` de sorte à le rendre exécutable. Vous exécuterez ensuite le fichier par `./install_tp2`. Si tout va bien, il devrait créer un sous répertoire tp2, télécharger les documents et données utiles. Vous disposerez ensuite d'une commande `notebook` qui permet de lancer le notebook ipython après avoir reconfiguré 2 variables d'environnement. 

![111](./Enluminure.png)

L'un des objectifs de ce TP est d'illustrer une méthode de discrimination, la **régression logistique**, sur des données internet réelles. Au passage, on verra également deux autres points importants dans l'analyse de données : 
- récupérer les données
- extraire, construire, des variables explicatives de ces données (*feature extraction*)

Les données auxquelles on s'intéressera sont *les réponses* fournies dans une base de type Stackoverflow. Plus exactement, on veut examiner des moyens d'évaluer la qualité des réponses fournies de manière automatique, avant même leur évaluation par des évaluateurs humains; éventuellement pour mettre en place une évaluation ou un guide automatique à la rédaction. Ici on ne prendra pas en compte le "sens", aussi l'exercice est particulièrement difficile, et il ne peut pas vraiment y avoir de miracle.  Voici quelques exemples de questions-réponses :

- [Exemple de question avec réponse négative](http://stackoverflow.com/questions/886955/breaking-out-of-nested-loops-in-java)

- [Un autre](http://stackoverflow.com/questions/3061/calling-a-function-of-a-module-from-a-string-with-the-functions-name-in-python)

- [Et encore une](http://stackoverflow.com/questions/9001509/how-can-i-sort-a-python-dictionary-sort-by-key)

- [Et une de plus](http://stackoverflow.com/questions/6797984/how-to-convert-string-to-lowercase-in-python)

A partir de là, voici quelques exemples des questions que l'on peut se poser :

- est-il possible de discriminer les réponses acceptées des non-acceptées ?
- est-il possible de discriminer les réponses correctes (note >0) des autres ?
- est-il possible de discriminer les réponses très bonnes réponses (note >10) des autres ?


Le problème est très difficile, la base est très bruitée. Voyons donc..

## Première partie : récupérer les données

Assez sympatiquement, Stackexchange fournit une API pour interroger ses bases. L'adresse et la documentation de l'API est :
[https://api.stackexchange.com/](https://api.stackexchange.com/)

Pour sélectionner des données à récupérer sur Stack, on peut configurer la requête via la page suivante :

[https://api.stackexchange.com/docs/advanced-search](https://api.stackexchange.com/docs/advanced-search)
Cette page permet de configurer le filtrage des données et les champs renvoyés. ceci est encodé dans le paramètre `filter` de la requête. 

La requête suivante permet ainsi de récupérer les 9 dernières questions, avec une réponse acceptée, à propos d'ipython :

https://api.stackexchange.com/docs/advanced-search#pagesize=9&order=desc&sort=activity&accepted=True&closed=True&tagged=ipython&filter=!*L1(ZTe*8)k0CMEL&site=stackoverflow&run=true 

> Faites le !

Les réponses peuvent être obtenues au format JSON (simplement en enlevant les deux dernières configs -- run et site). Par exemple, on utilisera

https://api.stackexchange.com/2.2/search/advanced?pagesize=9&order=desc&sort=activity&accepted=True&closed=True&tagged=ipython&site=stackoverflow&filter=!*L1(ZTe*8)k0CMEL

> Faites le !

Comme on le voit, le résultat est un fichier json, qui est similaire (mais pas totalement identique) à un dictionnaire Python. Pour récupérer des données, il suffit donc d'envoyer une série de reqêtes, de trier et d'enregistrer les réponses retournées. On se propose d'interrorger l'API avec le mot clé "python", entre deux dates, et de sauvegarder les réponses dans des fichiers csv. Commençons par le début : 

Le module `requests` est un module Python pratique qui permet d'envoyer des requêtes hhtp et de récupérer le résultat. 

> Consultez l'aide de `requests`

In [ ]:
import requests
# demandez l'aide de ce module

>- en utilisant le module `requests` (`requests.get`, attribut `content` ), récupérez les données associées à la requête 
https://api.stackexchange.com/2.2/search/advanced?fromdate=1422748800&todate=1423440000&order=desc&sort=activity&accepted=True&tagged=ipython&site=stackoverflow&filter=!SlE.x1mh.L6ZoGtJtT". Vous devrez décoder la réponse en utf8 en ajoutant un `.decode("utf8")`.  
Convertissez ensuite la réponse en dictionnaire à l'aide du module `json` (méthode `json.load`). Vous aurez besoin au passage d'utiliser `StringIO`, du module `io` qui permet d'associer un descripteur de fichier à un texte. 

In [ ]:
import requests
from io import StringIO 
import json

> Vous obtiendrez un dictionnaire, disons `dico_reponses`, que vous examinerez : afficher le dictionnaire, les clés, le contenu de la clé `items`, d'un élement de la clé items, par exemple `dico_reponses['items'][2]`

Pour faire varier les dates, il faut convertir celles-ci en epoch Unix. Cela peut se faire par : 

In [ ]:
import datetime
from datetime import timezone
print("date de maintenant: ", datetime.datetime.now())
print("convertie :",datetime.datetime.now().replace(tzinfo=timezone.utc).timestamp())
#Pour convertir une date quelconque :
print(datetime.datetime(2015,3,8).replace(tzinfo=timezone.utc).timestamp())

Sans enregistrement, il y une limite de 300 requêtes par jour. Pour disposer de quotas plus élevés, il faut demander une clé développeur, à l'adresse suivante : 
https://stackapps.com/users/login?returnurl=/apps/oauth/register.
La clé correspondant à l'utilisateur "Essais_ESIEE"  (Client Id 441) est 

> Key ACNpxD0PS7)lMe*lkPPOWw((

Ajouter un champ &key=ACNpxD0PS7)lMe*lkPPOWw((&  dans la requête.

Par ailleurs, Stackoverflow ne renvoie pas toutes les données, mais uniquement des "pages" (pour éviter les requêtes délirantes et la saturation de ses serveurs). La réponse contient une clé `has_more` qui si elle est True, dit qu'il faut passer à la page suivante. La requête peut contenir un paramètre `&page=`  qui permet d'indiquer cette page. Cette question n'est bien entendu pas essentielle, mais on vous invite à y réfléchir au moins un peu. 

>- Ecrire les quelques lignes qui permettraient de récupérer les données entre deux dates spécifiées (y,m, d), . Stocker le résultat dans une liste

In [ ]:
n=0
topic="perl"     # sujet de la requête (vous pouvez le changer !)
todate=int(datetime.datetime(2015,3,10).replace(tzinfo=timezone.utc).timestamp())
fromdate=int(datetime.datetime(2015,3,1).replace(tzinfo=timezone.utc).timestamp())
qadict=dict()     # Dictionnaire dans lequel on lira la réponse
stock=[]          # Liste dans laquelle on stockera les résultats
## Si ans est la réponse à la requête, on peut convertir le json en dictionnaire python par
# qadict=json.load(StringIO(ans))

In [ ]:
# A vous de compléter ici (décommenter le sligne sadéquates)
#qadict['has_more']=True
#while qadict['has_more']:
#    n=n+1
#    print("Itération {}".format(n))
    # A vous de compléter ici
    # qadict=json.load(StringIO(ans))
    # A vous de compléter ici
#print("Fini")    

Il faut ensuite stocker les résultats dans des fichiers. On se propose de le faire dans des fichiers de type csv. Une difficulté ici est que le nombre de réponses varie question par question, alors que le format d'un fichier csv est figé. Dans ce qui suit, on utilise un maximum de 6 réponses par question, initialisées à vide, et on stocke les différentes données. 

On va ensuite boucler sur le temps, en faisant varier les champs de date dans les requêtes, de manière à récupérer pas mal de données. On ne va pas vous le faire faire car cela prend pas mal de temps, à programmer comme à éxecuter (plusieurs jours), et ce n'est pas l'objectif principal du cours. 

Les fonctions suivantes sont donc fournies **à titre de documentation**. Les suggestions d'amélioration sont bienvenues. Vous êtes cependant invités à parcourir le code pour comprendre ce qui se passe. 



### Récupération sur plusieurs années

Pour écrire les données dans un fichier csv, nous utilisons  les fonctions suivantes :

```python
def cdico_init(nmax=6):
    """
    Initialise le dictionnaire dans lequel on va écrire les différents champs des réponses
    """
    cdico={}
    body_keys = ['answer{}_body'.format(ii) for ii in range(1,nmax)]
    score_keys =  ['answer{}_score'.format(ii) for ii in range(1,nmax)]
    nbcomments_keys =  ['answer{}_nbcomments'.format(ii) for ii in range(1,nmax)]
    for key in body_keys: cdico[key]=""
    for key in score_keys: cdico[key]=0
    for key in nbcomments_keys: cdico[key]=0
    cdico['accepted']=None    
    return cdico
```

```python
def save_qa(csvfilename,qa_dict,mode='wt',nmax=6):
    """
    Fonction de sauvegarde des réponses json en un fichier csv de nom "csvfilename". 
    Par défaut, ce fichier est ouvert en mode "wt", mode texte, écriture et écrase si 
    le fichier existe. le nombre maximal de réponses est nmax. on utilise la méthode 
    DictWriter du module csv pour écrire le fichier. 
    """
    import csv
    fieldnames=['creation_date','question_id', 'tags', 'question_title', 'question_body', 'answer_count', 'accepted']
    for n in range(nmax):
        fieldnames.append('answer{}_body'.format(n))
        fieldnames.append('answer{}_score'.format(n))
        fieldnames.append('answer{}_nbcomments'.format(n))

    csvfile=open(csvfilename, mode, encoding='utf8')    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,quoting=csv.QUOTE_ALL)
    if mode=='wt': 
        print("writing header")
        writer.writeheader()    

    for k in range(len(qa_dict['items'])):
        #print("*"*40)
        if qa_dict['items'][k]['is_answered']:
            cdico=cdico_init(nmax)
            cdico['creation_date']= qa_dict['items'][k]['creation_date']
            cdico['question_id']= qa_dict['items'][k]['question_id']
            cdico['tags']= qa_dict['items'][k]['tags']
            cdico['question_title']= qa_dict['items'][k]['title']
            cdico['question_body']= qa_dict['items'][k]['body']
            cdico['answer_count']= qa_dict['items'][k]['answer_count']
            try:
                answers=qa_dict['items'][k]['answers']        
            except:
                answers=[]
            for n,answer in enumerate(answers):
                if n>=nmax: break  
                if answer['is_accepted']:
                    cdico['accepted']=n
                cdico['answer{}_body'.format(n)] =  answer['body']
                cdico['answer{}_score'.format(n)] =  answer['score']
                cdico['answer{}_nbcomments'.format(n)] =  answer['comment_count']
            
            writer.writerow(cdico)    
    csvfile.close()        
```    

Boucle de lecture et de stockage des données sur plusieurs années (entre 2008 et 2015)

```python
import requests            
from io import StringIO 
import json
import datetime, time

#for topic in ('ipython', 'python'):
for topic in ['python']:
    #init date initiale
    todate=int(datetime.datetime(2008,12,1).replace(tzinfo=timezone.utc).timestamp()) #
    # Boucle sur années et mois ----------------------
    for year in range(2008,2015): # 2009
        for month in range(1,13):
            print("#"*60)
            print("topic", topic,"year",year)

            fromdate=todate #int(datetime.datetime(year,1,1).replace(tzinfo=timezone.utc).timestamp()) #2011
            todate=int(datetime.datetime(year,month,1).replace(tzinfo=timezone.utc).timestamp()) #2011
            #nom du fichier de sauvegarde
            csvfilename='qa_'+topic+'_'+str(datetime.date(year,month,1))+'.csv'
            #autres inits
            n=0
            qadict=dict()
            qadict['has_more']=True
            #
            # boucle sur les segnments de la réponse
            #
            while qadict['has_more']:
                n=n+1
                req="https://api.stackexchange.com/2.2/search/advanced?key=ACNpxD0PS7)lMe*lkPPOWw((&page={0}&fromdate={1}&todate={2}&order=desc&sort=activity&accepted=True&tagged={3}&site=stackoverflow&filter=!SlE.x1mh.L6ZoGtJtT".format(n,fromdate,todate,topic)
                r=requests.get(req)
                ans=r.content
                ans=ans.decode('utf8')
                qadict=json.load(StringIO(ans))
                        # save
                print("Saving shrink n=",n)
                mode='wt' if n==1 else 'at'
                save_qa(csvfilename,qadict,mode,nmax=6)
                time.sleep(5)
```

## Deuxième partie: Trouver et définir des features

Une étape très importante avant l'analyse des données proprement dite est d'extraire, de construire éventuellement, des variables qui permettront de distinguer les classes d'intérêt (variables explicatives). C'est ce qu'on se propose de faire dans cette partie. 

L'un des fichiers générés est fourni ici : `qa_python_2014-11-01.csv` (réponses à la requête "Python" pour le mois de novembre 2014. Uniquement les questions comportant une réponse acceptée. 14 Mo ! L'ensemble des fichiers représente une taille de 655 Mo ! Chaque ligne contient la question posée, six des réponses apportées, avec leurs scores respectifs et les textes des réponses. C'est sur les textes es réponses que l'on va travailler. 

### Lecture de données

>A l'aide du module pandas, il est particulièrement simple de charger un fichier csv. Chargez le fichier csv précédent, sous le nom df,  en utilisant la méthode `pd.read_csv`. Enumérer les colonnes et affichez la description par respectivement `df.columns` et `df.describe()` 

In [ ]:
import pandas as pd
# A vous de jouer ensuite
#...

Sélectionnons le texte de l'une des réponses

Pour sélectionner une donnée dans la dataframe `df`, on peut adresser d'abord la colonne, puis la ligne, suivant `df[label_de_la_colonne][index de la ligne]`.  
>Afficher le score et le texte de la première réponse (answer0) correspondant à la 21e question. Sauvegardez le texte de la réponse dans une variable `tst_texte`.

### Construction de variables explicatives 

A partir de ce texte, on veut construire différents indicateurs de "qualité". Des indicateurs de base seront bien-entendu

- la longueur du texte, 
- nombre de mots, 
- le nombre de lignes,
- le nombre de lignes de code,
- les références (liens internet),
- la mise en forme (gras, italique, etc).

On pourrait y ajouter

- nombre de mots en majuscule (mauvais style),
- nombre d'images,
- nombre de mots moyen par phrase...

Les suggestions sont bienvenues. 

Voici un exemple d'une telle fonction. 

In [ ]:
import re
def nb_paragraphs(text):
    nb_p=len(re.findall("<p>",text))
    return nb_p

##exemple
#nb_paragraphs(tst_texte)

> - Choisissez deux ou trois de ces caractéristiques et écrivez les fonctions permettant d'extraire les paramètres associés. 

Vous en profiterez pour manipuler les expressions régulières, qui ont été présentées (ou pas) lors du cours sur le [kit de survie](http://perso.esiee.fr/~bercherj/IT3007/Intro_Python.html#Expressions-régulières). L'exemple traité a été choisi en lien avec le problème qui nous occupe...

En utilisant `re.findall` vous devriez pouvoir extraire au moins le nombre d'images, le nombre de liens, de nombre de mots en majuscules. Pour le nombre de mots, il suffit de faire un `.split()`, un split sur les \n pour compter le nombre de lignes. Les autres sont un peu plus compliquées. 

In [ ]:
# fonction qui extrait calcule..
# A vous de faire
#

In [ ]:
# fonction qui extrait calcule..
# A vous de faire
#

Le corrigé fournit une classe `answer_metrics` qui permet de calculer tout un tas de métriques associées à un texte. 

>Importez la classe en question par 

>     from answer_metrics import *
>Consultez son aide par   

>      help(answer_metrics)

>Vous pouvez regarder comment elle est faite par 

>      %load anwer_metrics

> calculer le nombre de lignes total, de lignes de code, de nombre de liens sur la chaîne `tst_texte` 

In [ ]:
from answer_metrics import *

In [ ]:
# %load answer_metrics.py
import re
class answer_metrics():
    """
     Calcul de tout un tas de métriques associées à un texte. 
     'av_words' : nombre moyen de mots par phrase
     'code' :  code contenu dans le texte
     'extract_code' : métode pour extraire le code
     'html' :  texte initial
     'len_html' : longueur du html
     'nb_allcaps' : nombre de mots en majuscules
     'nb_codelines' : nombre de lignes de code
     'nb_imgs' : nombre d'images
     'nb_lines' : nombre de lignes
     'nb_links' : nombre de liens
     'nb_paragraphs' : nombre de paragraphes
     'nb_pretty' : nombre de mises en forme gras, italique, souligné,  
     'nb_words' : nombre de mots
     'strip_code' : méthode pour retirer le code
     'striphtml' : méthode pour retirer le texte html
     'text' : le texte sans html
     =================================================
     auteur: jfb mars 2015
    
    """
    import re
    def __init__(self,text=None):
        if text is None: 
            print("You  need to feed the object with a text")
        else:    
            self.html=text
            self.code=self.extract_code()
            self.text=self.striphtml(self.strip_code())
        
    def __call__(self,text=None):
        if text is None: 
            print("You  must feed the object with a text")
        else:    
            self.html=text
            self.code=self.extract_code()
            self.text=self.striphtml(self.strip_code())                    

    def striphtml(self,text):
        return re.sub('<[^<]+?>', '', text)
    
    def len_html(self):
        return len(self.html) 
    
    def extract_code(self):
        out=re.findall('<pre><code>([\s\S]*?)</code></pre>', self.html)
        return out
    
    def nb_paragraphs(self):
        nb_p=len(re.findall("<p>",self.html))
        return nb_p
    
    def strip_code(self):
        out=re.sub('<pre><code>([\s\S]*?)</code></pre>', '',self.html)
        return out

    def nb_codelines(self):
        Lcode=0
        for code in self.code:
            Lcode=Lcode+len(code.split('\n'))-1 #not perfect. Fails on expression involving \n
        return Lcode

    def nb_lines(self):
        Llines=len(self.text.split('\n'))
        return Llines

    def nb_allcaps(self):
        allcaps=re.findall('\\b[A-Z]{2,}\\b',self.text)
        Lallcaps=len(allcaps)
        return Lallcaps

    def nb_words(self):
        Nbwords=len(self.text.split())
        return Nbwords

    def av_words(self):
        sentences=re.split("\\w[\.!?]",self.text)
        nb_sentences=len(sentences)
        nb_words=0
        for sentence in sentences: nb_words+=len(sentence.split())
        return nb_words, nb_words/nb_sentences    
    
    def nb_pretty(self):
        return len(re.findall('<strong>',self.html))+\
                len(re.findall('<li>',self.html))+\
                len(re.findall('<em>',self.html))

    def nb_links(self):
        links=re.findall('<a href="http://.*?".*?>(.*?)</a>',self.html)
        nb_links=len(links)
        return nb_links

    def nb_imgs(self):
        imgs=re.findall('<img(.*?)/>',self.html)
        nb_imgs=len(imgs)
        return nb_imgs    

#a=answer_metrics (tst_texte)       

### Calcul des métriques pour l'ensemble des réponses de la base

Ceci en main, il est possible d'associer un ensemble de métriques à chacune des réponses. Il faut donc relire tous les fichiers csv dans lesquels on a stocké toutes les réponses ; et pour chacune calculer les métriques, noter le score obtenu par la réponse et si elle a été acceptée. Tout ceci sera ensuite conservé dans un grand tableau, pour usage ultérieur. 

Comme précédemment, on ne va pas vous le faire faire car cela prend pas mal de temps, et ce n'est pas l'objectif principal du cours. 

Les fonctions suivantes sont donc fournies **à titre de documentation**. Les suggestions d'amélioration sont bienvenues. Vous êtes cependant invités à parcourir le code pour comprendre ce qui se passe. 


Les deux routines suivantes font ce travail là. 

```python
def zaza(qa_file):
    for k in range(nb):
        accepted=df2['accepted'][k]
        if np.isnan(accepted): accepted=0
            
        ans_count=np.min([df2['answer_count'][k], 6]) # we just have stocked a maximum of 6 answers
        if np.isnan(accepted): accepted=0
        all_ans_html=[df2['answer{:d}_body'.format(n)][k] \
                               for n in range(ans_count)]
        all_ans_score=[df2['answer{:d}_score'.format(n)][k] \
                               for n in range(ans_count)]
        
        yield all_ans_html, all_ans_score, accepted
```    

Votre serviteur a fait un certain travail : récupéré toutes les questions réponses sur le thème "python", entre 2007 et 2015. Il a étiqueté tout cela sous la forme d'un énorme tableau. 

```python
import numpy as np

nb_features=10
BigA=np.empty((0,nb_features+2))
BigNA=np.empty((0,nb_features+2))

#Liste de tous les fichiers mensuels de 2008 à 2015
qa_list=['qa_python_{0:}-{1:02d}-01.csv'.format(y,m) for y in 
         [2007,2008,2009,2010, 2011, 2012,2013,2014,2015] for m in range(1,13)]

for qa_file in qa_list:
    print("Processing ",qa_file)
    try:
        df2=pd.read_csv(qa_file)
    except:
        print("read error")
        continue
        
    nb=df2.count()['answer_count']
    A=np.empty((5*nb,nb_features+2))
    NA=np.zeros((5*nb,nb_features+2)); #NA.fill(np.nan)
    nn=-1
    mm=-1
    
    for n,tt in enumerate(zaza(qa_file)):
        all_ans_html, all_ans_score, accepted = tt
        for k,m in enumerate(all_ans_html):
            print("k",k,"accepted", accepted)
            a=answer_metrics (m) 
            score= all_ans_score[k]
            accept=1 if k==accepted else 0
            print("accept",accept)
            if score>-10003:# was >20  or k==accepted:
                mm=mm+1
                A[mm,0:nb_features+2]=np.array([a.nb_allcaps(), a.nb_codelines(), a.nb_paragraphs(), 
                           a.nb_words(), a.nb_lines(), a.nb_links(), a.nb_imgs(), a.av_words()[1],
                           a.len_html(), a.nb_pretty(), score, accept])
            else:
                nn=nn+1
                NA[nn,0:nb_features+2]=np.array([a.nb_allcaps(), a.nb_codelines(), a.nb_paragraphs(), 
                   a.nb_words(), a.nb_lines(), a.nb_links(), a.nb_imgs(), a.av_words()[1],
                   a.len_html(), a.nb_pretty(), score, accept])
    
    if mm!=-1: BigA=np.concatenate((BigA, A[:mm,:]))        
    if nn!=-1: BigNA=np.concatenate((BigNA, NA[:nn,:]))  
```

In [ ]:
#Sauvegarde du résultat
"""
# au format pickle
import pickle
with open("A_all_withaccepted.pkl", "wb") as f: 
    pickle.dump(BigA,f)
# et
#au format csv en passant par pandas
Acols = ['nb_allcaps', 'nb_codelines', 'nb_paragraphs', 'nb_words', 'nb_lines', 
         'nb_links', 'nb_imgs', 'av_words', 'len_html', 'nb_pretty', 'score', 'accepted']
AA=pd.DataFrame(BigA), columns=Acols)
AA.to_csv("A_all_withaccepted.csv")
"""

## Troisième partie: discrimination

On va maintenant pouvoir effectuer les tâches de discrimination envisagées au début du texte. Pour cela on commence par importer la table et on analyse les principales caractéristiques de la chose. On créera ensuite une colonne cible 'target' selon le problème à traiter. 

> - Importer la table contenue dans le fichier "A_all_withaccepted.csv" par un `pd.read_csv(...)`. 
Regardez les principales statistiques par un `nom_variable.describe()`. 

In [ ]:
feature= ["allcaps", "nb_codelines", "nb_paragraphs", 
            "nb_words", "nb_lines", "nb_links", "nb_imgs", "av_words", "len_html", "pretty"]
cols=feature+['score', 'accepted']
#évidemment on ne met ni le score ni accepted dans les "features" sinon ce serait un peu facile...

On normalise les features, c'est-à-dire qu'on retire la moyenne et qu'on normalise par l'écart type

In [ ]:
"""
dataset=AA
##feature normalization - ça permet de gagner 1/2 %
for feat_col in cols[:-2]:
    m=dataset[feat_col].mean()
    s=dataset[feat_col].std()
    print(feat_col,m,s)
    #dotaset[feat_col]=dataset[feat_col].apply(lambda x: (x-m)/s)
    dataset[feat_col]=(dataset[feat_col]-m)/s
    dataset[feat_col+'2']=dataset[feat_col]**2#.apply(lambda x: x**2)
dataset.describe()   
"""

### Première question

- <span style="color:blue"> Est-il possible de discriminer les réponses acceptées des non-acceptées ?</span>

#### Travail préparatoire - représentation des données

Il n'est pas forcément nécessaire d'optimiser sur 400000 échantillons. On peut tirer au hasard un tableau plus petit. Cela se ferait ici de la manière suivante :

In [ ]:
import numpy as np
AAnew=AA.loc[np.random.choice(np.shape(AA)[0],size=50000, replace=False)]  # pour un dataframe
##Anew=A[np.random.choice(np.shape(A)[0],size=20000, replace=False),:] #pour un array

On ajoute une colonne target qu'on place à 1 si la réponse est acceptée à 0 sinon (identique à accepted et donc un peu superflu...)

> Le tableau sur lequel vous allez travailler est `dataset`. Ajouter une colonne 'target' que vous placerez à 1 si la réponse est acceptée à 0 sinon (identique à 'accepted' et donc un peu superflu, mais on aime bien que la cible s'appelle target...)

> Vous débuterez par une analyse graphique de la situation. On crée un tableau Pos qui contient les données pour la classe positive et un tableau Neg pour la classe négative.

In [ ]:
### Indices des réponses acceptées
I=AAnew['accepted']==1
Pos=AAnew[I]
Neg=AAnew[~I]

Voici un exemple de représentation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# Cette bibliothèque permet d'intégrer les images avec un support dynamique javascript 
#(bibliothèque D3.js) http://fr.wikipedia.org/wiki/D3.js
# ce qui permet des zooms dans la version html des pages comme dans le notebook. 
import mpld3
mpld3.enable_notebook()


In [ ]:
feature_name='nb_codelines'
rmax=100
plt.figure()
Pos[feature_name].hist(range=[0,rmax],bins=40,alpha=0.5,label="positive class")
Neg[feature_name].hist(range=[0,rmax],bins=40,alpha=0.5,label="negative class")
plt.title(feature_name)
_=plt.legend(loc="best")

>A partir de là, tracez l'ensemble des histogrammes pour chacun des features (faites une boucle). Vous pouvez récupérer la liste des features par `feature=list(Pos.columns)`.  Pour chacun des histogrammes, vous pouvez définir une valeur max pour l'abscisse.

In [ ]:
# Sympathiquement, on vous donne la liste des rmax pour les abscisses, sous la forme d'un dico
feature=list(Pos.columns)
feature=feature[1:-3]   #on supprime les trois dernières colonnes 
                        #qui sont le score, accepted et target
r=[15, 40, 40, 500, 80, 10, 10, 40, 800,10,40,2]
dico_rmax={ f:r[k] for k,f in enumerate(feature)}
print(dico_rmax)

> A vous de jouer pour les autres variables explicatives


#### Discrimination

Pour utiliser la régression logistique, nous aurons besoin de la classe `LogisticRegression`, d'une méthode découpant en un ensemble d'apprentissage et de test, `train_test_split` et éventuellement des méthodes d'analyse et rapport `confusion_matrix`, `classification_report`. 

Celles-ci sont importées par

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

>- Découper le tableau dataset en une table `features` (sélectionner les colonnes intéressantes, à l'aide de la liste des features `feature`), et une table `target` qui extrait la colonne cible. 
- définir une base de test et une base d'apprentissage en utilisant la méthode `train_test_split(features, target)`
- instancier la classe `LogisticRegression`, par exemple sous le nom `cls`, puis apprendre par la méthode `fit` et prédire par `predict`. 

>Vous pouvez voir quels sont les coefficients obtenus par `cls.intercept_` et `cls.coef_`.

On utilisera ensuite les deux routines maison `mycfm` et `classif_eval` pour évaluer les performances. Ces deux routines peuvent être chargées par 

    from mycfm import *
    from classif_eval import *
Vous pouvez également, dans le notebook, faire un 

    %load mycfm.py
ce qui vous permet de consulter le source (et donc l'aide).     

In [ ]:
from mycfm import *
from classif_eval import *

In [ ]:
%load mycfm.py

>Il vous faut donc générer les prédictions des classes. Celles-ci sont directement retournées par `predictions = cls.predict(features_test)`, pour un seuil de 0.5. Vous pouvez également ajuster vous même le seuil en utilisant la méthode `cls.predict_proba(features_test)[:,1]` dont vous comparerez la sortie à une proba seuil. 

> - générer la matrice de confusion. Examiner les performances obtenues. 

Donc en gros score de 60%, précision de 60% pour un recall un petit peu < 0.5. Ce n'est pas extraordinaire, mais pas insensé. 

>- Qu'obtiendrait-on en classant au hasard ? 

>En faisant varier le seuil de décision, on va examiner ce que l'on peut obtenir. Pour cela, vous utiliserez la fonction `classif_eval` qui rend les différents taux. 

>- Vous tracerez alors les courbes précision/recall/spécificité, le score, en fonction du seuil et enfin la courbe ROC. Vous pourrez utiliser ce qu'on a fait en cours [(ici)](http://perso.esiee.fr/~bercherj/IT3007/R%C3%A9gression_logistique.html#Evaluation-des-performances,-matrice-de-confusion) comme modèle. 

### Seconde question

On s'intéresse maintenant à une question un peu différente. Est-on capable de discriminer les réponses qui ont un peu d'intérêt, de score >0 des réponses pas bonnes (score < 0) ou pas intéressantes (score=0). 

- <span style="color:blue"> est-il possible de discriminer les réponses correctes (note >0) des autres ?</span>

C'est exactement la  même histoire que précédemment, on modifie simplement la cible `target` par le résultat d'un test sur le score. 

On le fait pour vous. Vous devez comprendre. 

In [ ]:
# sous échantillonnage de la grosse table
AAnew=AA.loc[np.random.choice(np.shape(AA)[0],size=100000, replace=False)]  # pour un dataframe
dataset=AAnew
# définition de la cible
dataset['target']=dataset['score']>0
# mapping car sklearn veut des nombres
dataset['target']=dataset['target'].map({True:1, False:0})
# on affiche le début de la table pour vérifier
dataset.head()

>- implanter une régression logistique pour ce problème,
- examinez la matrice de confusion, 
- tracer les courbes de perf. 

>Commenter sur les performances. Optimiser les paramètres pour trouver une précision aussi grande que possible, un taux de vrais négatifs > 40%, un recall (taux de vrais positifs) > 40%.

In [ ]:
# Régression logistique
#
# A vous de jouer


In [ ]:
# calcul et affichage de la matrice de confusion
#
# A vous de jouer


In [ ]:
#Courbes de performances
#
# A vous de jouer

In [ ]:
#Sélection d'un seuil et affichage de matrice de confusion correspondant aux perfs pour ce seuil
#
# A vous de jouer
